In [9]:
## Importing libraries
import pandas as pd
import numpy as np
import cv2
import os
import glob
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import sys

import tensorflow as tf
from tensorflow.keras import Sequential, models
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPool2D
from keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm


# Data preprocessing


In [1]:
#Encoding
label_code = {'mask_weared_incorrect':0,'with_mask':1,'without_mask':2}
label_decode = ['mask_weared_incorrect','with_mask','without_mask']

In [56]:
PATH = "D:\Mask detector\Dataset"
IMAGE_SIZE = (224, 224)
EPOCHS = 70
BATCH_SIZE = 128
NUM_CLASSES = 3

In [58]:
train_gen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split = 0.2)

val_gen = ImageDataGenerator(validation_split=.2)


In [59]:
SEED = 24
train_data = train_gen.flow_from_directory(
    PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    seed=SEED,
    subset='training'
)

val_data = val_gen.flow_from_directory(
    PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    seed=SEED,
    shuffle=False,
    subset='validation'
)

Found 7188 images belonging to 3 classes.
Found 1794 images belonging to 3 classes.


# Building CNN


In [61]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)

16705208/16705208 [==============================] - 5s 0us/step


In [62]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [63]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

early_stopping = EarlyStopping(patience=5, verbose=1)
checkpoint = ModelCheckpoint('mask_model.h5', save_best_only=True, verbose=1)
lr_reduce = ReduceLROnPlateau(patience=2, verbose=1)

In [64]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(amsgrad=True),
    loss='categorical_crossentropy',
#     our data classes are perfectly balanced so I'm able to use simple accuracy metric
    metrics=['acc']
)

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    callbacks=[early_stopping, checkpoint, lr_reduce]
)

Epoch 1/70
